In [1]:
# Module imports and auto-reload setup
%load_ext autoreload
%aimport if_lib, if_utils, if_dpp, if_graphics, if_consts, if_gc1dpp
%autoreload 1
import os
import json
import random

from if_utils import get_filename, show_data, save_traces

from if_lib import generate_random_challenge, read_HMAC, read_keypair, get_id_person, get_location_id, \
get_unit_id, get_resource_spec_id, get_resource, get_process, create_event, make_transfer, reduce_resource, set_user_location

## Configuration and Endpoints

In [2]:
# Define constants for this use case
USE_CASE = 'Lauds_Machine-3D-Printer'

# # Zenflows API endpoint
# ENDPOINT = 'https://proxy.dpp-staging.dyne.im/zenflows/'

# # DPP service endpoint
# DPP_URL = 'https://proxy.dpp-staging.dyne.im/'

# Zenflows API endpoint
ENDPOINT = 'https://proxy.dpp-staging.dyne.im/zenflows/api'

# DPP service endpoint
DPP_URL = 'https://proxy.dpp-staging.dyne.im/interfacer-dpp'

# All participants
USERS = ["A","B"]

## File Path Configuration

In [3]:
# Calculate names of settings files
USERS_FILE = get_filename('cred_users.json', ENDPOINT, USE_CASE)
LOCS_FILE = get_filename('loc_users.json', ENDPOINT, USE_CASE)
UNITS_FILE = get_filename('units_data.json', ENDPOINT, USE_CASE)
SPECS_FILE = get_filename('res_spec_data.json', ENDPOINT, USE_CASE)
DPP_FILE = get_filename('dpp_data.json', ENDPOINT, USE_CASE)
RES_FILE = get_filename('initial_resources.json', ENDPOINT, USE_CASE)
PROCESS_FILE = get_filename('process_data.json', ENDPOINT, USE_CASE)

print(f"Data will be saved to:")
print(f"  Users: {USERS_FILE}")
print(f"  Locations: {LOCS_FILE}")
print(f"  Units: {UNITS_FILE}")
print(f"  Resource Specs: {SPECS_FILE}")
print(f"  DPP Data: {DPP_FILE}")
print(f"  Initial Resources: {RES_FILE}")
print(f"  Processes: {PROCESS_FILE}")

Data will be saved to:
  Users: use_cases/Lauds_Machine-3D-Printer/proxy.dpp-staging.dyne.im%2Fzenflows%2Fapi/cred_users.json
  Locations: use_cases/Lauds_Machine-3D-Printer/proxy.dpp-staging.dyne.im%2Fzenflows%2Fapi/loc_users.json
  Units: use_cases/Lauds_Machine-3D-Printer/proxy.dpp-staging.dyne.im%2Fzenflows%2Fapi/units_data.json
  Resource Specs: use_cases/Lauds_Machine-3D-Printer/proxy.dpp-staging.dyne.im%2Fzenflows%2Fapi/res_spec_data.json
  DPP Data: use_cases/Lauds_Machine-3D-Printer/proxy.dpp-staging.dyne.im%2Fzenflows%2Fapi/dpp_data.json
  Initial Resources: use_cases/Lauds_Machine-3D-Printer/proxy.dpp-staging.dyne.im%2Fzenflows%2Fapi/initial_resources.json
  Processes: use_cases/Lauds_Machine-3D-Printer/proxy.dpp-staging.dyne.im%2Fzenflows%2Fapi/process_data.json


## Initialize Data Structures

In [4]:
# Create data structures
process_data = {}
res_data = {}
event_seq = []
dpp_data = {}

if os.path.isfile(USERS_FILE):
    with open(USERS_FILE,'r') as f:
        users_data = json.loads(f.read())
    print("Credentials file available for users")
else:
    users_data = {}
    users_data['A'] = {
      "userChallenges": {
        "whereParentsMet": "London",
        "nameFirstPet": "Fuffy",
        "nameFirstTeacher": "Jim",
        "whereHomeTown": "Paris",
        "nameMotherMaid": "Wright"
      },
      "name": "User A",
      "username": "userA_username",
      "email": "userA@example.org",
      "note": "me.userA.org"
    }
    users_data['B'] = {
        "userChallenges": {
            "whereParentsMet":"Amsterdam",
            "nameFirstPet":"Toby",
            "nameFirstTeacher":"Juliet",
            "whereHomeTown":"Rome",
            "nameMotherMaid":"Banks"
        },
        "name": "User B",
        "username": "userB",
        "email": "userB@example.org",
        "note" : "me.userB.org"
    }

    with open(USERS_FILE,'w') as f:
        json.dump(users_data, f)

# Initialize or load location data
if os.path.isfile(LOCS_FILE):
    with open(LOCS_FILE,'r') as f:
        locs_data = json.loads(f.read())
    print("Location file available")
else:
    locs_data = {}
    locs_data['A'] = {
        "name": "OLVG",
        "lat": 52.35871773455108,
        "long": 4.916762398221842,
        "addr": "Oosterpark 9, 1091 AC Amsterdam",
        "note": "location.user1.org"
    }
    locs_data['B'] = {
        "name": "CleanLease",
        "lat" : 51.47240440868687,
        "long" : 5.412460440524406,
        "addr" : "De schakel 30, 5651 Eindhoven",
        "note": "location.user2.org"
    }
    with open(LOCS_FILE,'w') as f:
        json.dump(locs_data, f)


if os.path.isfile(UNITS_FILE):
    with open(UNITS_FILE,'r') as f:
        units_data = json.loads(f.read())
    print(f"Unit file available")
else:
    units_data = {}
#     with open(file,'w') as f:
#         json.dump(units_data, f)

# Initialize units and specs
if os.path.isfile(UNITS_FILE):
    with open(UNITS_FILE,'r') as f:
        units_data = json.loads(f.read())
    print(f"Unit file available")
else:
    units_data = {}

if os.path.isfile(SPECS_FILE):
    with open(SPECS_FILE,'r') as f:
        res_spec_data = json.loads(f.read())
    print(f"Resource Spec file available")
else:
    res_spec_data = {}

## Authentication Setup: HMAC Generation

In [5]:
# Read HMAC or get it from the server
for user in USERS:
    read_HMAC(USERS_FILE, users_data, user, endpoint=ENDPOINT)

Payload
{'query': 'mutation ($firstRegistration: Boolean!, $userData: JSONObject!){\n  \n        keypairoomServer(firstRegistration: $firstRegistration, userData: $userData)\n      \n      }', 'variables': {'firstRegistration': True, 'userData': '{"email": "userA@example.org"}'}}
Variables
{'firstRegistration': True, 'userData': '{"email": "userA@example.org"}'}
Result
<Response [200]>
JSON
{
  "data": null,
  "errors": [
    {
      "message": "email exists",
      "path": [
        "keypairoomServer"
      ],
      "locations": [
        {
          "line": 3,
          "column": 9
        }
      ]
    }
  ]
}
Payload
{'query': 'mutation ($firstRegistration: Boolean!, $userData: JSONObject!){\n  \n        keypairoomServer(firstRegistration: $firstRegistration, userData: $userData)\n      \n      }', 'variables': {'firstRegistration': False, 'userData': '{"email": "userA@example.org"}'}}
Variables
{'firstRegistration': False, 'userData': '{"email": "userA@example.org"}'}
Result
<Resp

## Cryptographic Key Generation

In [6]:
# Read the keypair
for user in USERS:
    read_keypair(USERS_FILE, users_data, user)

result: ZenResult(output='{"ecdh_public_key":"BFPu8vnqtTZON1Hy18knEqBBbe6qQt6KIEWUftzMo2LxVitLGZ7cF0iGmqfMRdn+3QUh0W2Jx+ff0l8L3KlSSss=","eddsa_public_key":"b3XYnrMAPYre61K2jH6gq3A4G6xZrKYfuZ9kwihs55S","ethereum_address":"0x840dC4D3a8Eb7a637461E7d7f4255f88a401aE28","keyring":{"ecdh":"eoFzf1cKHG+F7Q+fRVZck4b5bty+44DuNlyMvhXcgLI=","eddsa":"DCqKRyK14C3Jmk4ifpGgMQFcv8nznouynSYXA8RMRieC","ethereum":"64dc1f10597f9d449d3f86f6813b994c20d9993118876202e2e510abb2839d94","reflow":"qiPrpyDcuzR4TIGrTww9QQpjh5BTIakAaBtDzx+ZcGs=","schnorr":"R2Oyqv+hZpzr3tN2Lva6BveecL/23Ekv6nbIKMMPuw8="},"reflow_public_key":"BVOLDa8kfM2TYF4i5uuglgWNURQS8YP3gpT9Em3aXCpbWgMtJB5yPOSgMz6bwnXKBqZN7UZL+CCn7ZMJNY6rOfaEABq4a8H4acWla1lTR7PSd5b1/Gbci1Q5TIpxbUraFzSsoZUy2v4jZFeo0WryaPJ1pEcTPnBVS7OrF7d7riqx3XZ4xGaH/EjVOZLmD28fEZskHGd9OO0Z2tHfGkf3oKyC0Hfwjc1GL2T2D59XxOHE8QmehLlAHT3waF0G7hC7","schnorr_public_key":"EunSJfSzE2qz561tlTVBLK6nepu7IJ6Piw2AxMxdln3CWVdh896u6brWILGqspua","seed":"solution garage know special trap wheel timber r

## User Registration in Zenflows

In [7]:
# Read or get id of the person
for user in USERS:
    get_id_person(USERS_FILE, users_data, user, endpoint=ENDPOINT)

## Location Registration and Assignment

In [8]:
# Read or get the location id
for user in USERS:
    get_location_id(LOCS_FILE, users_data[user], locs_data, user, endpoint=ENDPOINT)
    set_user_location(USERS_FILE, users_data, locs_data, user, endpoint=ENDPOINT)

In [9]:
# Read or get the location id
for user in USERS:
    get_location_id(LOCS_FILE, users_data[user], locs_data, user, endpoint=ENDPOINT)
    set_user_location(USERS_FILE, users_data, locs_data, user, endpoint=ENDPOINT)

Location id available for OLVG
Location id available for User A
Location id available for CleanLease
Location id available for User B


## Unit of Measurement Registration

In [10]:
# Get the ids of all units
get_unit_id(UNITS_FILE, users_data['A'], units_data, 'piece', 'u_piece', 'om2:one', endpoint=ENDPOINT)
get_unit_id(UNITS_FILE, users_data['A'], units_data, 'mass', 'kg', 'om2:kilogram', endpoint=ENDPOINT)
get_unit_id(UNITS_FILE, users_data['A'], units_data, 'volume', 'lt', 'om2:litre', endpoint=ENDPOINT)
get_unit_id(UNITS_FILE, users_data['A'], units_data, 'time', 'h', 'om2:hour', endpoint=ENDPOINT)
get_unit_id(UNITS_FILE, users_data['A'], units_data, 'energy', 'Wh', 'om2:wattHour', endpoint=ENDPOINT)

## Process Definition

In [11]:
# Create the processes

# Create the process that wraps creating the 3D object “Luffy”
process_name = 'Create_3D_object_Luffy'
user_data = users_data['A']
note = f"3D printing process to create object 'Luffy' performed by {user_data['name']}"
get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)

# Save process data to file
with open(PROCESS_FILE, 'w') as f:
    json.dump(process_data, f, indent=2)
print(f"Process data saved to {PROCESS_FILE}")

Process data saved to use_cases/Lauds_Machine-3D-Printer/proxy.dpp-staging.dyne.im%2Fzenflows%2Fapi/process_data.json


## Resource Specification Registration

In [12]:
# Read all the resource specifications for 3D printing workflow

# 3D Printer
name = 'Machine-3D-Printer'
note = 'Specification for the 3D printer used by the operator'
classification = 'https://www.wikidata.org/wiki/Q2095'  # Wikidata: 3D printer
default_unit_id = units_data['piece']['id']
get_resource_spec_id(SPECS_FILE, users_data['A'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

# Filament
name = 'Filament'
note = 'Specification for 3D printing filament material'
classification = 'https://www.wikidata.org/wiki/Q11782'  # Wikidata: plastic (or adjust to specific filament type)
default_unit_id = units_data['mass']['id']
get_resource_spec_id(SPECS_FILE, users_data['A'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

# Electrical Energy
name = 'Electrical-Energy'
note = 'Specification for electrical energy consumed during 3D printing'
classification = 'https://www.wikidata.org/wiki/Q6837'  # Wikidata: electricity
default_unit_id = units_data['energy']['id']
get_resource_spec_id(SPECS_FILE, users_data['A'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

# 3D Object “Luffy”
name = '3D-object-Luffy'
note = 'Specification for the 3D printed object named Luffy'
classification = 'https://www.wikidata.org/wiki/Q1248784'  # Wikidata: manufactured object
default_unit_id = units_data['piece']['id']
get_resource_spec_id(SPECS_FILE, users_data['A'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

# G-code file specification
name = '3D-gcode-file-Luffy'
note = 'Specification for the 3D printing G-code file for Luffy'
classification = 'https://www.wikidata.org/wiki/Q1073076'  # Wikidata: Computer file
default_unit_id = units_data['piece']['id']
get_resource_spec_id(SPECS_FILE, users_data['A'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)


## Initial Resource Creation

In [13]:
# We create the resources that will not be saved to file as it is assumed they are recreated at each run

# 3D printer will be used
res_name = 'Machine-3D-Printer'
amount = 1
get_resource(res_data, res_spec_data, res_name, users_data['A'], event_seq, amount, endpoint=ENDPOINT)

# Filament for printing
res_name = 'Filament'
amount = 1  # in kg (adjust based on your workflow)
get_resource(res_data, res_spec_data, res_name, users_data['A'], event_seq, amount, endpoint=ENDPOINT)

# Electrical energy for printing
res_name = 'Electrical-Energy'
amount = 1000  # in Wh (adjust based on your workflow)
get_resource(res_data, res_spec_data, res_name, users_data['A'], event_seq, amount, endpoint=ENDPOINT)

# 3D-gcode file
res_name = '3D-gcode-file-Luffy'
amount = 1
get_resource(res_data, res_spec_data, res_name, users_data['A'], event_seq, amount, endpoint=ENDPOINT)

# Save initial resources to file
with open(RES_FILE, 'w') as f:
    json.dump(res_data, f, indent=2)
print(f"Initial resources saved to {RES_FILE}")

Initial resources saved to use_cases/Lauds_Machine-3D-Printer/proxy.dpp-staging.dyne.im%2Fzenflows%2Fapi/initial_resources.json


## Component Production

Now we'll produce all the components needed for the bike assembly.

### Step 1: consume for filament

In [14]:
# Define event consume for filament
cur_res = action = event_note = amount = cur_pros = None
action = 'consume'
event_note = 'consume filament for 3D-object-Luffy'
amount = 50  # example: 50 grams, adjust as needed
cur_pros = process_data['Create_3D_object_Luffy']

cur_res = res_data['Filament']  # pre-existing

event_id, ts = create_event(
    provider=users_data['A'],
    action='consume',                   # action on existing resource
    note='consume filament for 3D-object-Luffy',
    amount=50,
    process=process_data['Create_3D_object_Luffy'],
    res_spec_data=res_spec_data,
    existing_res=cur_res,              # use existing_res, NOT new_res
    endpoint=ENDPOINT
)
event_seq.append({'ts': ts, 'event_id': event_id, 'action': action, 'res_name': cur_res['name'], 'res': cur_res['id']})


### Step 2: Consume electricity

In [15]:
# Define event consume for electrical energy
cur_res = action = event_note = amount = cur_pros = None
action = 'consume'
event_note = 'consume electrical energy for 3D-object-Luffy'
amount = 500  # example: 500 Wh, adjust as needed
cur_pros = process_data['Create_3D_object_Luffy']

cur_res = res_data['Electrical-Energy']

event_id, ts = create_event(users_data['A'], action, event_note, amount=amount, process=cur_pros,
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id': event_id, 'action': action, 'res_name': cur_res['name'], 'res': cur_res['id']})


### Step 3: Cite 3d files

In [16]:
# Define event cite for 3D gcode file
cur_res = action = event_note = amount = cur_pros = None
action = 'cite'
event_note = 'cite 3D-gcode-file-Luffy'
amount = 1
cur_pros = process_data['Create_3D_object_Luffy']

cur_res = res_data['3D-gcode-file-Luffy']

event_id, ts = create_event(users_data['A'], action, event_note, amount=amount, process=cur_pros,
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id': event_id, 'action': action, 'res_name': cur_res['name'], 'res': cur_res['id']})


### Step 4:produce 3D-object-Luffy

In [17]:
# Define event produce for 3D object “Luffy”
cur_res = action = event_note = amount = cur_pros = None
action = 'produce'
event_note = 'produce 3D-object-Luffy'
amount = 1
cur_pros = process_data['Create_3D_object_Luffy']

res_data['3D-object-Luffy'] = {
    "res_ref_id": f'3D-object-Luffy-{random.randint(0, 10000)}',
    "name": '3D-object-Luffy',
    "spec_id": res_spec_data['3D-object-Luffy']['id']
}

print(res_data['3D-object-Luffy'])
cur_res = res_data['3D-object-Luffy']

event_id, ts = create_event(users_data['A'], action, event_note, amount=amount, process=cur_pros,
                 res_spec_data=res_spec_data, new_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id': event_id, 'action': action, 'res_name': cur_res['name'], 'res': cur_res['id']})
event_seq.append({'ts': ts, 'process_id':cur_pros['id'], 'name' : cur_pros['name']})

{'res_ref_id': '3D-object-Luffy-5595', 'name': '3D-object-Luffy', 'spec_id': '06E0HEN30JT66WW95TAZZDYNG0'}


## Save All Component Data

Save all produced components to the resources file so they can be loaded in the production notebook.

In [18]:
# Update the resources file with all components
with open(RES_FILE, 'w') as f:
    json.dump(res_data, f, indent=2)
print(f"✓ All resources (raw materials + components) saved to {RES_FILE}")
print(f"  - {len(res_data)} resources total")

✓ All resources (raw materials + components) saved to use_cases/Lauds_Machine-3D-Printer/proxy.dpp-staging.dyne.im%2Fzenflows%2Fapi/initial_resources.json
  - 5 resources total


## Setup Complete - Summary

In [19]:
print("\n" + "="*60)
print("SETUP COMPLETE")
print("="*60)
print(f"\n✓ {len(users_data)} users registered")
print(f"✓ {len(locs_data)} locations created")
print(f"✓ {len(units_data)} units registered")
print(f"✓ {len(res_spec_data)} resource specifications created")
print(f"✓ {len(process_data)} processes defined")
print(f"✓ {len(res_data)} resources created (raw materials + components)")
print(f"\nAll data saved to JSON files in: use_cases/{USE_CASE}/")
print(f"\nComponents produced:")
print(f"\nYou can now run the production notebook to create the bike with DPP.")


SETUP COMPLETE

✓ 2 users registered
✓ 2 locations created
✓ 5 units registered
✓ 5 resource specifications created
✓ 1 processes defined
✓ 5 resources created (raw materials + components)

All data saved to JSON files in: use_cases/Lauds_Machine-3D-Printer/

Components produced:

You can now run the production notebook to create the bike with DPP.
